In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 46.1 MB/s 
     |████████████████████████████████| 182 kB 79.0 MB/s 


In [ ]:
import json
from pathlib import Path
import torch
from torch.utils.data import DataLoader
import time

In [ ]:
# Define the bert tokenizer
from transformers import AutoTokenizer,AdamW,BertForQuestionAnswering, AutoModel
tokenizer = AutoTokenizer.from_pretrained("Rifky/Indobert-QA")

# Load the fine-tuned modeol
model = torch.load("/content/drive/MyDrive/Colab Notebooks/BERT/Model_Terjemahan_Arbain/AlWafi_e(5),l(0,00001),b(8)",map_location=torch.device('cpu'))
#model = AutoModel.from_pretrained("Rifky/Indobert-QA")
model.eval()

Downloading:   0%|          | 0.00/532 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/234k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31923, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
#fungsi prediksi

def predict(context,query):

  inputs = tokenizer.encode_plus(query, context, return_tensors='pt')

  outputs = model(**inputs)
  answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
  answer_end = torch.argmax(outputs[1]) + 1

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

  return answer

def normalize_text(s):
  """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
  import string, re

  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)

  def white_space_fix(text):
    return " ".join(text.split())

  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()

  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)

  common_tokens = set(pred_tokens) & set(truth_tokens)

  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0

  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)

  return 2 * (prec * rec) / (prec + rec)

def compute_precision(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()

  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)

  common_tokens = set(pred_tokens) & set(truth_tokens)

  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0

  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)

  return prec

def compute_recall(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()

  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)

  common_tokens = set(pred_tokens) & set(truth_tokens)

  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0

  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)

  return rec

In [ ]:
def give_an_answer(context,query,answer):
  prediction = predict(context,query)
  em_score = compute_exact_match(prediction, answer)
  f1_score = compute_f1(prediction, answer)
  prec = compute_precision(prediction, answer)
  rec = compute_recall(prediction, answer)

  print(f"Question: {query}")
  print(f"Prediction: {prediction}")
  print(f"True Answer: {answer}")
  print(f"EM: {em_score}")
  print(f"F1: {f1_score}")
  print(f"Precision: {prec}")
  print(f"Recall: {rec}")
  print("\n")

QUESTION ANSWERING

In [ ]:
context = "dari umar radhiyallahu 'anhu, dia berkata, aku mendengar rasulullah shallallahu 'alaihi wa sallam bersabda, 'sesungguhnya setiap perbuatan tergantung niatnya. dan sesungguhnya setiap orang (akan dibalas) berdasarkan apa yang dia niatkan. siapa yang hijrahnya karena (ingin mendapatkan keridhaan) allah dan rasul-nya, maka hijrahnya kepada (keridhaan) allah dan rasul-nya. dan siapa yang hijrahnya karena dunia yang dikehendakinya atau karena wanita yang ingin dinikahinya maka hijrahnya (akan bernilai sebagaimana) yang diniatkan'. (hr. al-bukhari dan muslim)."

queries = ["setiap perbuatan manusia tergantung dari apa?",
           "bagaimana jika hijrahnya seseorang adalah karena ingin mendapatkan keridhaan allah dan rasul-nya?",
           "bagaimana jika hijrahnya seseorang adalah karena dunia?"
           ]
answers = ["niatnya",
           "maka hijrahnya kepada (keridhaan) allah dan rasul - nya",
           "maka hijrahnya (akan bernilai sebagaimana) yang diniatkan"
           ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari umar radhiyallahu 'anhu juga berkata, 'ketika kami duduk-duduk di sisi rasulullah shallallahu 'alaihi wa sallam suatu hari tiba-tiba datanglah seorang laki-laki yang mengenakan baju yang sangat putih dan berambut sangat hitam, tidak tampak padanya bekas-bekas perjalanan jauh dan tidak ada seorangpun di antara kami yang mengenalnya. hingga kemudian dia duduk di hadapan nabi lalu menempelkan kedua lututnya kepada lutut beliau seraya berkata, 'wahai muhammad, beritahukan kepadaku tentang islam?' rasulullah shallallahu 'alaihi wa sallam menjawab, 'islam adalah engkau bersaksi bahwa tidak ada ilah yang berhak diibadahi selain allah, dan bahwa muhammad adalah utusan allah, engkau mendirikan shalat, menunaikan zakat, berpuasa di bulan ramadhan, dan pergi haji jika mampu'. kemudian orang itu berkata, 'engkau benar'. kami semua merasa heran kepadanya, dia yang menanyakan tetapi dia pula yang membenarkannya. kemudian dia bertanya lagi, 'beritahukan kepadaku tentang iman?' rasulullah shallallahu 'alaihi wa sallam menjawab, 'engkau beriman kepada allah, malaikat-malaikat-nya, kitab-kitab-nya, rasul-rasul-nya, hari akhir, dan engkau beriman kepada takdir yang baik maupun yang buruk'. lagi-lagi orang itu berkata, 'engkau benar'. selanjutnya dia berkata lagi, 'beritahukan kepadaku tentang ihsan?' rasulullah bersabda, 'ihsan adalah engkau beribadah kepada allah seakan-akan engkau melihatnya, jika engkau tidak melihatnya maka dia melihatmu'. kemudian orang itu berkata, 'beritahukan aku tentang hari kiamat (kapan terjadinya)?' beliau bersabda, 'yang ditanya tidak lebih tau dari yang bertanya'. orang itu bertanya lagi 'beritahukan kepadaku tanda-tandanya?' beliau menjawab, 'jika seorang hamba melahirkan tuannya, jika engkau melihat orang-orang tidak beralas kaki dan tidak berpakaian, miskin, dan penggembala domba namun mereka berlomba-lomba meninggikan bangunannya'. kemudian orang itu pergi dan aku terdiam sebentar. kemudian rasulullah shallallahu 'alaihi wa sallam bertanya, 'wahai umar, tahukah kamu siapa yang bertanya tadi?' aku berkata, 'allah dan rasul-nya lebih mengetahui'. beliau bersabda, 'dia adalah jibril yang datang kepada kalian untuk mengajarkan agama kepada kalian'. (hr. muslim)"

queries = ["siapakah yang bertanya kepada rasulullah tentang islam, iman, ihsan, dan hari kiamat?"]
answers = ["jibril"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abu abdurrahman, abdullah bin umar bin al-khathtab radhiyallahu 'anhu dia berkata, 'aku mendengar rasulullah shallallahu 'alaihi wa sallam bersabda, 'islam dibangun di atas lima perkara, yaitu kesaksian bahwa tiada ilah yang berhak diibadahi kecuali allah dan bahwa muhammad utusan allah, menegakkan shalat, menunaikan zakat, melaksanakan haji ke baitullah, dan puasa ramadhan'. (hr. al-bukhari dan muslim)"

queries = ["islam dibangun atas berapa perkara?"]
answers = ["lima"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abu abdurrahman abdullah bin mas'ud radhiyallahu 'anhu dia berkata: rasulullah shallallahu 'alaihi wa sallam beliau adalah orang yang benar dan dibenarkan, menyampaikan kepada kami: 'sesungguhnya setiap kalian dikumpulkan penciptaannya di perut ibunya sebagai setetes mani selama empat puluh hari, kemudian berubah menjadi setetes darah selama empat puluh hari, kemudian menjadi segumpal daging selama empat puluh hari. kemudian diutus kepadanya seorang malaikat, lalu ditiupkan padanya ruh dan diperintahkan untuk menetapkan empat perkara: menetapkan rezekinya, ajalnya, amalnya, dan kecelakaan atau kebahagiaannya. demi allah yang tidak ada ilah selain-nya, sesungguhnya di antara kalian ada yang melakukan perbuatan ahli surga hingga jarak antara dirinya dan surga tinggal sehasta, akan tetapi nasibnya telah didahului oleh kitab (takdir), dia melakukan perbuatan ahli neraka maka masuklah dia ke dalam neraka. sesungguhnya pula di antara kalian ada yang melakukan perbuatan ahli neraka hingga jarak antara dirinya dan neraka tinggal sehasta akan tetapi nasibnya telah didahului oleh kitab (takdir), lalu dia melakukan perbuatan ahli surga maka masuklah dia ke dalam surga'. (riwayat al-bukhari dan muslim)"

queries = ["setetes darah selama empat puluh hari akan berubah menjadi apa?",
           "bagaimanakah jika seseorang melakukan perbuatan ahli neraka?"
           ]
answers = ["segumpal daging",
           "maka masuklah dia ke dalam neraka"
           ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari ummul mukminin, ummu abdillah, aisyah radhiyallahu 'anha dia berkata, rasulullah shallallahu 'alaihi wa sallam bersabda, 'barangsiapa yang melakukan dalam urusan (agama) kami ini hal baru yang bukan berasal darinya, maka ia tertolak''. (hr. al-bukhari dan muslim), dalam riwayat muslim disebutkan, 'barangsiapa yang melakukan suatu perbuatan yang tidak berdasarkan perintah dari (agama) kami, maka ia tertolak'."

queries = ["bagaimana jika seseorang melakukan perbuatan yang tidak berdasarkan perintah agama?"]
answers = ["maka ia tertolak"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abu abdillah nu'man bin basyir radhiyallahu 'anhuma dia berkata, aku mendengar rasulullah shallallahu 'alaihi wa sallam bersabda, 'sesungguhnya yang halal itu jelas dan yang haram itu jelas. di antara keduanya terdapat perkara-perkara syubhat (samar-samar) yang tidak diketahui oleh kebanyakan orang. siapa yang menjaga diri dari perkara-perkara syubhat, maka dia telah membersihkan agama dan kehormatannya. dan siapa yang terjerumus dalam perkara syubhat, maka akan terjerumus dalam perkara yang diharamkan. sebagaimana penggembala yang menggembalakan hewan gembalaannya di sekitar daerah terlarang, hampir saja dia terjerumus memasukinya. ketahuilah bahwa setiap raja memiliki daerah larangan dan daerah larangan allah adalah apa yang dia haramkan. ketahuilah bahwa dalam tubuh manusia itu terdapat segumpal daging, jika segumpal daging itu baik maka baiklah seluruh tubuhnya, tetapi jika ia buruk, maka buruklah seluruh tubuhnya. ketahuilah bahwa segumpal daging itu adalah hati'. (hr. al-bukhari dan muslim)"

queries = ["bagaimana jika seseorang menjaga diri dari perkara syubhat?",
           "bagaimana jika segumpal daging yang didalam tubuh manusia itu baik?",
           "apa maksud dari segumapl daging itu?"
           ]
answers = ["maka dia telah membersihkan agama dan kehormatannya",
           "maka baiklah seluruh tubuhnya",
           "hati"
           ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari ibnu umar radhiyallahu 'anhuma sesungguhnya rasulullah shallallahu 'alaihi wa sallam bersabda, 'aku diperintahkan untuk memerangi manusia hingga mereka bersaksi bahwa tidak ada ilah yang berhak diibadahi kecuali allah dan bahwa muhammad adalah rasulullah, menegakkan shalat, dan menunaikan zakat. jika mereka melakukan hal itu, berarti mereka telah melindungi darah dan harta mereka dariku kecuali berdasarkan hak islam, dan perhitungan mereka ada pada allah ta'ala'. (hr. al-bukhari dan muslim)"

queries = ["siapa yang diperintahkan untuk memerangi manusia hingga mereka bersaksi bahwa  tidak ada ilah yang berhak diibadahi kecuali allah dan bahwa muhammad adalah rasulullah, menegakkan shalat, dan menunaikan zakat?"]
answers = ["rasulullah shallallahu ' alaihi wa sallam"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abu hurairah radhiyallahu 'anhu dia berkata: rasulullah shallallahu 'alaihi wa sallam bersabda: 'sesungguhnya allah ta'ala itu baik, dia tidak menerima kecuali yang baik. sesungguhnya allah memerintahkan orang beriman dengan apa yang juga dia perintahkan kepada para rasul-nya. allah berfirman, 'wahai para rasul, makanlah yang baik-baik dan kerjakanlah amal shalih' (al-mu'minun [23]: 51) dia juga berfirman, 'hai orang-orang yang beriman, makanlah di antara rezeki yang baik-baik yang kami berikan kepadamu' (al-baqarah [2]: 172) kemudian beliau menyebutkan ada seseorang melakukan perjalanan jauh, keadaannya kusut rambutnya dan berdebu tubuhnya, menengadahkan kedua tangannya ke langit seraya berdoa, 'ya rabbi!' sedangkan makanannya haram, minumannya haram, pakaiannya haram, dan dia dikenyangkan dari sesuatu yang haram. maka bagaimana doanya akan dikabulkan?'. (hr. muslim)"

queries = ["bagaimana jika seseorang berdoa tetapi minumannya haram?"]
answers = ["maka bagaimana doanya akan dikabulkan"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari ibnu mas'ud radhiyallahu 'anhu dia berkata, rasulullah shallallahu 'alaihi wa sallam bersabda, 'tidak halal darah seorang muslim kecuali dengan tiga sebab: orang yang sudah menikah lalu berzina, membunuh nyawa dibalas nyawa, dan orang yang meninggalkan agamanya serta berpisah dari jamaah (muslimin)'. (hr. al-bukhari dan muslim)"

queries = ["tidak halal darah seorang muslim kecuali dengan berapa sebab?"]
answers = ["tiga"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abu hurairah radhiyallahu 'anhu, sesungguhnya rasulullah shallallahu 'alaihi wa sallam bersabda, 'barangsiapa beriman kepada allah dan hari akhir hendaklah dia berkata yang baik atau diam, barangsiapa beriman kepada allah dan hari akhir hendaklah dia menghormati tetangganya, dan barangsiapa beriman kepada allah dan hari akhir hendaklah dia memuliakan tamunya'. (hr. al-bukhari dan muslim)"

queries = ["orang yang beriman kepada allah dan hari akhir hendaklah dia memuliakan siapa?"]
answers = ["tamunya"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abu hurairah radhiyallahu 'anhu, bahwa seseorang berkata kepada rasulullah shallallahu 'alaihi wa sallam, 'berilah aku pesan'. beliau bersabda, 'kamu jangan marah'. orang itu mengulangi perkataannya berkali-kali namun beliau hanya menjawab, 'kamu jangan marah'. (hr. bukhari)"

queries = ["berapa kali orang itu mengulangi perkataannya?"]
answers = ["berkali - kali"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abu ya'la syaddad bin aus radhiyallahu 'anhu, dari rasulullah shallallahu 'alaihi wa sallam, beliau bersabda, 'sesungguhnya allah telah memerintahkan untuk berbuat baik dalam segala sesuatu. jika kalian membunuh maka lakukanlah dengan baik. jika kalian menyembelih maka lakukanlah dengan baik pula, kalian tajamkan pisaunya dan kalian buat senang hewan sembelihannya'. (hr. muslim)"

queries = ["siapa yang telah menetapkan perbuatan baik?",
           "bagaimana jika kita ingin membunuh sesuatu?"
           ]
answers = ["allah",
           "maka lakukanlah dengan baik"
           ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abu dzar jundub bin junadah dan abu abdurrahman mu'adz bin jabal radhiyallahu 'anhuma, dari rasulullah shallallahu 'alaihi wa sallam, beliau bersabda, 'bertakwalah kepada allah di mana pun kalian berada, ikutilah amal keburukan dengan amal kebaikan, niscaya amal kebaikan akan menghapus amal keburukan, dan pergaulilah manusia dengan akhlak yang baik'. (hr. at-tirmidzi, dan beliau berkata, 'hadits ini hasan', pada sebagian naskah yang dikatakan, 'hasan shahih')"

queries = ["dimana kita harus bertakwa kepada allah?"]
answers = ["di mana pun kalian berada"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abu al-abbas abdullah bin abbas radhiyallahu 'anhuma, dia berkata, 'suatu saat saya membonceng nabi shallallahu 'alaihi wa sallam maka beliau bersabda, 'wahai ananda, aku akan mengajarkan kepadamu beberapa perkara: jagalah allah, niscaya dia akan menjagamu. jagalah allah niscaya dia akan selalu berada di hadapanmu. jika kamu meminta, maka mintalah hanya kepada allah. jika kamu memohon pertolongan, maka mohonlah pertolongan hanya kepada allah. ketahuilah jika suatu umat berkumpul untuk memberi manfaat kepadamu, mereka tidak akan dapat memberikan manfaat sedikit pun kecuali apa yang telah allah tetapkan bagimu. dan jika mereka berkumpul untuk mencelakakanmu, mereka tidak akan dapat melakukannya kecuali jika kecelakaan itu telah allah tetapkan bagimu. pena telah diangkat dan lembaran telah kering'. (hr. at-tirmidzi dan beliau berkata, 'hadits ini hasan shahih'. dalam riwayat selain at-tirmidzi disebutkan: 'jagalah allah, pasti kamu mendapati-nya di hadapanmu. ingatlah allah di waktu lapang (senang), niscaya allah akan mengingatmu di waktu sempit (susah). ketahuilah bahwa apa yang semestinya tidak menimpa kamu, maka hal itu tidak akan menimpamu, dan apa yang semestinya menimpamu tidak akan terhindar darimu. ketahuilah sesungguhnya kemenangan menyertai kesabaran dan sesungguhnya kesenangan menyertai kesusahan dan kesulitan')"

queries = ["kepada siapa kita memohon pertolongan?"]
answers = ["allah"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abu malik al-harits bin 'ashim al-'asy'ari radhiyallahu 'anhu, dia berkata, rasulullah shallallahu 'alaihi wa sallam bersabda, 'bersuci sebagian dari iman, ucapan alhamdulillah dapat memenuhi timbangan (amal), ucapan subhanallah wal hamdulillah dapat memenuhi antara langit dan bumi, shalat adalah cahaya, sedekah adalah bukti, al-qur'an dapat menjadi saksi yang meringankanmu atau yang memberatkanmu. semua manusia berangkat di pagi hari menjual dirinya, ada yang membebaskan dirinya (dari kehinaan dan adzab) ada pula yang menghancurkan dirinya'. (hr. muslim)"

queries = ["bersuci merupakan sebagian dari apa?",
           "apa yang termasuk sebagian dari iman?",
           "ucapan subhanallah walhamdulillah dapat apa?",
           "apa yang dapat memenuhi ruang antara langit dan bumi?"
           ]
answers = ["iman",
           "bersuci",
           "memenuhi antara langit dan bumi",
           "ucapan subhanallah wal hamdulillah"
           ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abu dzar al-ghifari radhiyallahu 'anhu, dari rasulullah shallallahu 'alaihi wa sallam, tentang apa yang beliau riwayatkan dari rabb-nya 'azza wa jalla, 'wahai hamba-ku, sesungguhnya aku telah mengharamkan kezhaliman atas diri-ku dan begitu pula aku jadikan haram di antara kalian, maka janganlah kalian saling menzhalimi. wahai hamba-ku kalian semua tersesat kecuali orang yang aku beri hidayah, maka mintalah hidayah kepada-ku niscaya aku akan memberikan kalian hidayah. wahai hamba-ku, kalian semuanya kelaparan kecuali orang yang aku berikan kepadanya makanan, maka mintalah makanan kepada-ku niscaya aku berikan kalian makan. wahai hamba-ku, kalian semua telanjang kecuali orang yang aku berikan kepadanya pakaian, maka mintalah pakaian kepada-ku niscaya aku berikan kalian pakaian. wahai hamba-ku kalian selalu saja berbuat kesalahan pada malam dan siang hari, dan aku-lah yang mengampuni dosa semuanya, maka mintalah ampunan kepada-ku niscaya akan aku ampuni kalian. wahai hamba-ku sesungguhnya kalian tidak dapat memberi kemudharatan bagi-ku sebagaimana kalian tidak dapat memberi kemanfaatan kepada-ku. wahai hamba-ku, seandainya seluruh makhluk yang pertama kali diciptakan hingga yang terakhir dari kalangan manusia dan jin berada dalam keadaan ketakwaan sebagaimana orang yang paling bertakwa di antara kalian, niscaya hal tersebut tidak menambah kerajaan-ku sedikit pun. wahai hamba-ku, seandainya seluruh makhluk yang pertama kali diciptakan hingga yang terakhir dari kalangan manusia dan jin berada dalam kedurhakaan sebagaimana orang yang paling durhaka di antara kalian, niscaya hal itu tidak mengurangi kerajaan-ku sedikit pun. wahai hamba-ku, seandainya seluruh makhluk yang pertama kali diciptakan hingga yang terakhir berdiri di sebuah lembah lalu mereka meminta kepada-ku, lalu setiap orang yang meminta aku penuhi niscaya hal itu tidak mengurangi apa yang ada pada-ku kecuali bagaikan sebuah jarum yang dicelupkan ke tengah lautan. wahai hamba-ku, sesungguhnya seluruh perbuatan kalian akan dihisab (hitung) kemudian diberikan balasannya, maka barangsiapa yang mendapati amalnya berupa kebaikan hendaklah dia bersyukur kepada allah dan siapa yang mendapatinya selain dari itu maka janganlah dia mencela kecuali terhadap dirinya sendiri'. (hr. muslim)"

queries = ["siapakah orang yang tidak lapar?"]
answers = ["orang yang aku berikan kepadanya makanan"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abu dzar radhiyallahu 'anhu, dia mengatakan bahwa beberapa orang dari shahabat datang menemui rasulullah shallallahu 'alaihi wa sallam dan berkata, 'wahai rasulullah, orang-orang kaya telah unggul dengan membawa pahala yang banyak, mereka shalat sebagaimana kami shalat, mereka puasa sebagaimana kami puasa dan mereka bersedekah dengan kelebihan harta mereka'. maka beliau bersabda, 'bukankah allah telah memberikan kepada kalian cara lain untuk bersedekah? sesungguhnya setiap ucapan tasbih dihitung sedekah, setiap ucapan takbir dihitung sedekah, setiap ucapan tahmid dihitung sedekah, setiap ucapan tahlil dihitung sedekah, beramar makruf nahi mungkar dihitung sedekah, dan pada perbuatan kalian menggauli istri dihitung sedekah'. mereka bertanya, 'wahai rasulullah, betulkah seseorang di antara kami menyalurkan syahwatnya akan mendapatkan pahala?' beliau bersabda, 'menurut kalian, seandainya syahwatnya tersebut disalurkan di jalan yang haram bukankah dia berdosa? demikianlah halnya jika syahwatnya dia letakkan pada jalan yang halal, maka dia mendapatkan pahala'. (hr. muslim)"

queries = ["siapa orang yang dimaksud membawa pahala yang banyak?",
           "ucapan tasbih merupakan apa?",
           "bagaimana jika syahwatnya dia letakkan pada jalan yang halal?"
           ]
answers = ["orang - orang kaya",
           "sedekah",
           "maka dia mendapatkan pahala"
           ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abu hurairah radhiyallahu 'anhu, dia berkata bahwa rasulullah shallallahu 'alaihi wa sallam bersabda, 'masing-masing persendian tubuh manusia ada kewajiban untuk disedekahi setiap hari di waktu matahari terbit: engkau mendamaikan dua orang yang bertikai adalah sedekah, engkau membantu seseorang yang kerepotan dengan hewan tunggangannya, lalu engkau membantunya mengangkatkan barang ke atas punggung hewan tunggangan adalah sedekah, ucapan yang baik adalah sedekah, setiap langkah yang engkau ayunkan untuk mendatangi shalat berjamaah adalah sedekah, dan menghilangkan gangguan dari jalan adalah sedekah'. (hr. al-bukhari dan muslim)"

queries = ["setiap persendian anggota tubuh manusia wajib untuk apa?",
           "membantu seseorang yang berkendara lalu mengangkatkan barangnya termasuk apa?"
           ]
answers = ["disedekahi",
           "sedekah"
           ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari an-nawas bin sam'an radhiyallahu 'anhu dari rasulullah shallallahu 'alaihi wa sallam, beliau bersabda, 'kebaikan adalah akhlak yang terpuji, dosa adalah apa yang meresahkan jiwamu dan engkau tidak suka jika hal itu diketahui oleh manusia'. (hr. muslim)"

queries = ["apa itu kebaikan?"]
answers = ["akhlak yang terpuji"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari mu'adz bin jabal radhiyallahu 'anhu, dia berkata, saya bertanya, 'wahai rasulullah, baritahu saya tentang perbuatan yang dapat memasukkanku ke dalam surga dan menjauhkanku dari neraka'. beliau bersabda, 'engkau telah bertanya tentang sesuatu yang besar, dan perkara tersebut akan menjadi mudah bagi orang yang dimudahkan oleh allah ta'ala, yaitu: hendaklah engkau beribadah kepada allah dan tidak menyekutukan-nya dengan sesuatu apapun, menegakkan shalat, menunaikan zakat, puasa ramadhan, dan pergi haji'. kemudian beliau bersabda, 'maukah engkau aku beritahu tentang pintu-pintu surga? puasa itu perisai, sedekah memadamkan dosa sebagaimana air memadamkan api, begitu pula shalatnya seseorang di tengah malam (qiyamullail). kemudian beliau membacakan ayat: 'lambung mereka jauh dari tempat tidurnya...' (as-sajadah [32]: 16-17). kemudian beliau bersabda, 'maukah engkau aku beri tahu tentang pokok dari segala perkara serta tiangnya dan puncaknya?' saya menjawab, 'mau ya rasulullah'. beliau bersabda, 'pokok segala perkara adalah islam, tiangnya adalah shalat, dan puncaknya adalah jihad'. kemudian beliau bersabda kembali, 'maukah engkau aku beritahu pengendali semua itu?' saya menjawab, 'mau ya rasulullah'. beliau memegang lisannya lalu bersabda, 'jagalah ini'. saya berkata, 'wahai rasulullah, apakah kita akan dihukum juga atas apa yang kita bicarakan?' beliau bersabda, 'ibumu kehilangan kamu, menurutmu adakah yang menyebabkan seseorang jatuh tersungkur di atas wajahnya di dalam neraka (atau dalam riwayat lain: tersungkur di atas hidungnya) kalau bukan karena buah dari ucapannya'. (hr. at-tirmidzi dan dia berkata, 'hadits ini hasan shahih')"

queries = ["apa pokok dari segala perkara?"]
answers = ["islam"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abi tsa'labah al-khusyani jurtsum bin nasyir radhiyallahu 'anhu, dari rasulullah shallallahu 'alaihi wa sallam dia berkata: 'sesungguhnya allah ta'ala telah menetapkan beberapa kewajiban, maka janganlah kalian mengabaikannya. dia telah menetapkan batasan-batasan, maka janganlah kalian melampauinya. dia telah mengharamkan beberapa perkara, maka janganlah kalian melanggarnya. dia mendiamkan beberapa hal sebagai bentuk rahmat (kasih sayang) bagi kalian dan bukan karena lupa, maka janganlah kalian mencari-cari tentangnya (menanyakannya)'. (hr. qaruquthni dan lainnya)"

queries = ["siapakah yang telah menetapkan beberapa kewajiban?"]
answers = ["allah ta ' ala"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abu sa'id al-khudri radhiyallahu 'anhu berkata, 'saya mendengar rasulullah shallallahu 'alaihi wa sallam bersabda, 'barangsiapa yang melihat kemungkaran hendaknya ia mengubahnya dengan tangannya, jika tidak mampu hendaknya ia mengubahnya dengan lisannya, jika tidak mampu hendaknya ia mengubah dengan hatinya, namun itulah selemah-lemah iman'. (hr. muslim)"

queries = ["bagaimana jika seseorang melihat kemungkaran?"]
answers = ["hendaknya ia mengubahnya dengan tangannya"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abu hurairah radhiyallahu 'anhu, dari rasulullah shallallahu 'alaihi wa sallam bersabda, 'barangsiapa yang meringankan sebuah kesulitan (derita) dunia dari seorang mukmin, niscaya allah akan meringankan untuknya sebuah kesulitan di hari kiamat. barangsiapa yang memberikan kemudahan bagi orang yang sedang mengalami kesulitan, niscaya akan allah membantu memudahkannya saat mengalami kesulitan di dunia dan akhirat. barangsiapa yang menutupi (aib) seorang muslim, allah akan tutupkan aibnya di dunia dan akhirat. allah selalu menolong hamba-nya, selama hamba-nya itu menolong saudaranya. siapa yang menempuh jalan untuk mendapatkan ilmu, maka allah akan memudahkan baginya jalan ke surga dengannya. tidaklah sebuah kaum itu berkumpul di salah satu rumah allah, membaca kitab-kitab allah dan saling mengajarkan di antara mereka, melainkan akan diturunkan kepada mereka ketenangan, akan diselubungkan kepada mereka rahmat, mereka akan dikelilingi malaikat, dan allah akan menyebut-nyebut mereka di hadapan para makhluk yang di sisi-nya. dan siapa yang lambat amalnya, maka ia tidak akan dapat dipercepat oleh nasabnya'. (diriwayatkan oleh muslim dengan redaksi seperti ini)"

queries = ["apa yang akan diperoleh jika seseorang menutub aib seorang muslim?"]
answers = ["allah akan tutupkan aibnya di dunia dan akhirat"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari ibnu abbas radhiyallahu 'anhuma, dari rasulullah shallallahu 'alaihi wa sallam sebagaimana yang beliau riwayatkan dari rabbnya yang mahasuci dan mahatinggi, 'sesungguhnya allah telah menetapkan kebaikan dan keburukan, kemudian menjelaskan hal tersebut: 'siapa yang ingin melaksanakan kebaikan kemudian dia tidak mengamalkannya, maka allah mencatatnya di sisi-nya sebagai satu kebaikan yang sempurna. dan jika dia berniat melakukannya dan kemudian melaksanakannya, maka allah akan mencatatnya di sisi-nya sebagai sepuluh kebaikan hingga tujuh ratus kali lipat bahkan hingga kelipatan yang banyak. dan jika dia berniat melaksanakan keburukan kemudian dia tidak melaksanakannya, maka allah mencatat di sisi-nya sebagai satu kebaikan yang sempurna. adapun jika dia berniat kemudian melaksanakannya, maka allah mencatatnya sebagai satu keburukan saja'. (hr. al-bukhari dan muslim dalam kedua shahihnya dengan redaksi seperti ini)"

queries = ["bagaimana jika seseorang berniat melakukan kebaikan kemudian tidak melaksanakannya?",
           "bagaimana jika seseorang berniat melakukan keburukan dan kemudian melaksanakannya?"
           ]
answers = ["maka allah mencatatnya di sisi - nya sebagai satu kebaikan yang sempurna",
           "maka allah mencatatnya sebagai satu keburukan saja"
           ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari abu hurairah radhiyallahu 'anhu berkata bahwasanya rasulullah shallallahu 'alaihi wa sallam bersabda, 'sesungguhnya allah ta'ala berfirman, 'siapa yang memusuhi wali-ku maka aku telah mengumumkan perang kepadanya. tidaklah hamba-ku mendekatkan diri kepada-ku dengan sesuatu yang lebih aku sukai melebihi beribadah dengan apa yang telah aku wajibkan kepadanya. dan hamba-ku masih terus mendekatkan diri kepada-ku dengan nawafil (ibadah-ibadah sunnah di luar yang fardhu) hingga aku pun mencintainya dan jika aku telah mencintainya maka aku adalah pendengarannya yang dia gunakan untuk mendengar, penglihatannya yang dia gunakan untuk melihat, tangannya yang digunakannya untuk memukul dan kakinya yang digunakan untuk berjalan. jika dia meminta kepada-ku, niscaya akan aku berikan dan jika dia minta perlindungan dari-ku, niscaya aku akan benar-benar melindunginya'. (hr. bukhari)"

queries = ["bagaimana jika seseorang meminta perlindungan kepada allah?"]
answers = ["niscaya aku akan benar - benar melindunginya"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari ibnu umar radhiyallahu 'anhuma berkata, 'rasulullah shallallahu 'alaihi wa sallam pernah memegang kedua pundak saya seraya bersabda, 'jadilah engkau di dunia seakan-akan orang asing atau pengembara'. ibnu umar berkata, 'jika kamu berada di sore hari jangan tunggu pagi hari, dan jika kamu berada di pagi hari jangan tunggu sore hari, gunakanlah kesehatanmu untuk (persiapan saat) sakitmu dan kehidupanmu untuk kematianmu'. (hr. bukhari)"

queries = ["gunakanlah kehidupanmu untuk apa?"]
answers = ["kematianmu"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)
#-------------------------------------------------------------------------------
context = "dari anas radhiyallahu 'anhuma dia berkata: saya mendengar rasulullah shallallahu 'alaihi wa sallam bersabda, 'allah ta'ala berfirman: 'wahai anak adam, selama engkau mau berdoa dan berharap kepada-ku, pasti aku akan memberikan ampunan untukmu atas segala dosa yang telah kamu lakukan dan aku tidak peduli (berapapun banyaknya dan besarnya dosamu). wahai anak adam, seandainya dosa-dosamu bertumpuk hingga mencapai awan di langit lalu kamu memohon ampun kepada-ku, niscaya aku akan mengampunimu. wahai anak adam, seandainya engkau datang kepada-ku dengan kesalahan (dosa) sepenuh bumi, kemudian engkau menjumpai-ku dalam kondisi tidak menyekutukan-ku sedikit pun, maka aku akan datang kepadamu  dengan ampunan sepenuh (bumi) itu pula'. (riwayat at-tirmidzi dan dia berkata: haditsnya hasan shahih)"

queries = ["apa yang allah sampaikan jika seseorang berdoa dan berharap kepada - nya?"]
answers = ["pasti aku akan memberikan ampunan untukmu"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)

Question: setiap perbuatan manusia tergantung dari apa?
Prediction: niatnya
True Answer: niatnya
EM: 1
F1: 1.0
Precision: 1.0
Recall: 1.0


Question: bagaimana jika hijrahnya seseorang adalah karena ingin mendapatkan keridhaan allah dan rasul-nya?
Prediction: maka hijrahnya kepada ( keridhaan ) allah dan rasul - nya.
True Answer: maka hijrahnya kepada (keridhaan) allah dan rasul - nya
EM: 1
F1: 1.0
Precision: 1.0
Recall: 1.0


Question: bagaimana jika hijrahnya seseorang adalah karena dunia?
Prediction: maka hijrahnya ( akan bernilai sebagaimana ) yang diniatkan '
True Answer: maka hijrahnya (akan bernilai sebagaimana) yang diniatkan
EM: 1
F1: 1.0
Precision: 1.0
Recall: 1.0


Question: siapakah yang bertanya kepada rasulullah tentang islam, iman, ihsan, dan hari kiamat?
Prediction: umar radhiyallahu ' anhu
True Answer: jibril
EM: 0
F1: 0
Precision: 0
Recall: 0


Question: islam dibangun atas berapa perkara?
Prediction: lima
True Answer: lima
EM: 1
F1: 1.0
Precision: 1.0
Recall: 1.0


Q